<a href="https://colab.research.google.com/github/Pevb1014/DatosAbiertos/blob/main/Procesamiento_De_Datos_Pablo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Procesamiento PR Señal Horizontal.csv**

In [ ]:
# Verificar que la cantidad de separadores en todos los registros sea igual

bad_rows = []
with open("/content/drive/MyDrive/Documentos/Pablo/Proyectos/Datos abiertos/Datos crudos/Otros Datos Crudos/PR Señal Horizontal.csv", "r", encoding="utf-8", errors="ignore") as f:
    for i, line in enumerate(f, start=1):
        num_commas = line.count(",")
        bad_rows.append((i, num_commas))

# calcular moda de comas normales
from statistics import mode
expected_commas = mode([c for _, c in bad_rows])

# buscar filas que se salen del estándar
errores = [(i, c) for i, c in bad_rows if c != expected_commas]

print("Filas problemáticas:")
for i, c in errores[:]:
    print(f"Línea {i}: tiene {c} comas (esperadas {expected_commas})")


In [ ]:
#Cargar el dataset y ver informacion basica de su contenido

df = pd.read_csv("/content/drive/MyDrive/Documentos/Pablo/Proyectos/Datos abiertos/Datos crudos/Otros Datos Crudos/PR Señal Horizontal.csv")
df.info()


In [ ]:
#Eliminacion de las columnas sin relevancia

df = df.drop(columns=["objectid", "codigo", "ubicacion", "clasificac", "tipo_pintu", "territoria", "shape_leng", "z", "st_length(shape)"])

In [ ]:
#Manejo de valores nulos en cada columna
#Eliminacion de registros que no tienen pr_inicial y pr_final a la vez
#Eliminar regisros sin via

df = df[~(
    (df["via"].isna() | (df["via"].str.strip() == "")) |
    (df["pr_inicial"].isna() & df["pr_final"].isna())
)]

#Si pr_inicial es nulo y es mayor a 0 se coloca: pr_inicial = pr_final - 1
mask = df["pr_inicial"].isna() & df["pr_final"].notna()

df.loc[mask, "pr_inicial"] = df.loc[mask, "pr_final"].apply(
    lambda x: x-1 if x > 0 else 0
)
#Si pr_final es nulo se coloca: pr_final = pr_inicial 1 1
mask2 = df["pr_final"].isna() & df["pr_inicial"].notna()

df.loc[mask2, "pr_final"] = df.loc[mask2, "pr_inicial"].apply(
    lambda x: x + 1
)


In [ ]:
#Obtencion del año del contrato
df["numero_contrato"] = df["contrato"].str.split("_").str[0]
df["anio_contrato"]   = df["contrato"].str.split("_").str[1]
df.drop(columns=["contrato"], inplace=True)


In [ ]:
#Descarga dataset procesado a archivo excel
df.to_excel("PR Señal Horizontal Limpio.xlsx", index=False)

**Procesamiento Postes Referencia**

In [ ]:
# Verificar que la cantidad de separadores en todos los registros sea igual

bad_rows = []
with open("/content/drive/MyDrive/Documentos/Pablo/Proyectos/Datos abiertos/Datos crudos/Otros Datos Crudos/Postes Referencia.csv", "r", encoding="utf-8", errors="ignore") as f:
    for i, line in enumerate(f, start=1):
        num_commas = line.count(",")
        bad_rows.append((i, num_commas))

# calcular moda de comas normales
from statistics import mode
expected_commas = mode([c for _, c in bad_rows])

# buscar filas que se salen del estándar
errores = [(i, c) for i, c in bad_rows if c != expected_commas]

print("Filas problemáticas:")
for i, c in errores[:20]:
    print(f"Línea {i}: tiene {c} comas (esperadas {expected_commas})")


In [ ]:
#Cargar el dataset y ver informacion basica de su contenido

df = pd.read_csv("/content/drive/MyDrive/Documentos/Pablo/Proyectos/Datos abiertos/Datos crudos/Otros Datos Crudos/Postes Referencia.csv")
df.info()


In [ ]:
df = df.drop(columns=["objectid", "distancia", "lado", "material", "fechainstalacion", "fuente", "fechafuente", "altura", "estadociclovida"])
df["postereferencia"] = pd.to_numeric(df["postereferencia"], errors="coerce")
df["estado"] = pd.to_numeric(df["estado"], errors="coerce").astype("Int64")
df["latitud"] = pd.to_numeric(df["latitud"], errors="coerce")
df["longitud"] = pd.to_numeric(df["longitud"], errors="coerce")
df["observacion"] = df["observacion"].fillna("").astype(str).str.strip()
df = df.dropna(subset=["latitud", "longitud"])

#Verificar que la longitud y latitud estan en el rango de las de Colombia
df = df[
    (df["latitud"].between(-4.3, 13.5)) &
    (df["longitud"].between(-82, -66))
]
#Agregar la columna maps asociando link a la ubicacion
df["maps"] = (
    '=HYPERLINK("https://www.google.com/maps?q=' +
    df["latitud"].astype(str) + "," +
    df["longitud"].astype(str) +
    '", "Ver en Maps")'
)

In [ ]:
df.to_excel("Postes Referencia Limpio.xlsx", index=False)


**Procesamiento PR Señal Vertical.csv**

In [ ]:
# Verificar que la cantidad de separadores en todos los registros sea igual

bad_rows = []
with open("/content/drive/MyDrive/Documentos/Pablo/Proyectos/Datos abiertos/Datos crudos/Otros Datos Crudos/PR Señal Vertical.csv", "r", encoding="utf-8", errors="ignore") as f:
    for i, line in enumerate(f, start=1):
        num_commas = line.count(",")
        bad_rows.append((i, num_commas))

# calcular moda de comas normales
from statistics import mode
expected_commas = mode([c for _, c in bad_rows])

# buscar filas que se salen del estándar
errores = [(i, c) for i, c in bad_rows if c != expected_commas]

print("Filas problemáticas:")
for i, c in errores[:20]:
    print(f"Línea {i}: tiene {c} comas (esperadas {expected_commas})")

In [ ]:
#Cargar el dataset y ver informacion basica de su contenido

df = pd.read_csv("/content/drive/MyDrive/Documentos/Pablo/Proyectos/Datos abiertos/Datos crudos/Otros Datos Crudos/PR Señal Vertical.csv")
df.info()


In [ ]:
#Eliminacion de las columnas sin relevancia

df = df.drop(columns=["objectid", "codigo", "lado", "tipo_senal", "codigo_sen", "forma_sena", "territoria", "estado", "limpieza", "cimiento", "soporte", "visibilida", "mat_placa", "sustento", "z"])

In [ ]:
#Eliminar regisros sin via o sin pr

df = df[~(
    (df["via"].isna() | (df["via"].str.strip() == "")) |
    df["pr"].isna()
)]

# Transformar fecha_insp de timestamp en milisegundos a fecha de Colombia
df["fecha_insp"] = (
    pd.to_datetime(df["fecha_insp"], unit="ms", utc=True)
      .dt.tz_convert("America/Bogota")
      .dt.date
)

#Obtencion del año del contrato
df["numero_contrato"] = df["contrato"].str.split("_").str[0]
df["anio_contrato"]   = df["contrato"].str.split("_").str[1]
df.drop(columns=["contrato"], inplace=True)

In [ ]:
df.to_excel("PR Señal Vertical Limpio.xlsx", index=False)

Procesamiento SIV Señal Vertical

In [ ]:
#Verificar lineas con columnas estra o faltantes
import csv

csv_file = "/content/drive/MyDrive/Documentos/Pablo/Proyectos/Datos abiertos/Datos crudos/Datos Limpios/SIV Señal Vertical.csv"

with open(csv_file, encoding="utf-8") as f:
    reader = csv.reader(f)
    rows = list(reader)

# Número de columnas que debería tener cada registro
expected_cols = len(rows[0])

broken_rows = []

for i, row in enumerate(rows):
    if len(row) != expected_cols:
        broken_rows.append((i+1, row))

print("Filas con problemas:")
for idx, r in broken_rows:
    print(f" - Línea {idx}: {r}")


In [ ]:
#Identificar registros divididos en varias lineas
import string

csv_file = "/content/drive/MyDrive/Documentos/Pablo/Proyectos/Datos abiertos/Datos crudos/Datos Limpios/SIV Señal Vertical.csv"

with open(csv_file, encoding="utf-8") as f:
    lines = f.readlines()

broken = []

for i, line in enumerate(lines):
    raw = line.rstrip("\n")
    stripped = raw.strip()

    # Saltar encabezado (línea 1)
    if i == 0:
        continue

    # --- REGLAS DE DETECCIÓN ---

    # 1. Línea vacía
    if stripped == "":
        broken.append((i+1, raw))
        continue

    # 2. Línea que comienza con coma
    if stripped.startswith(","):
        broken.append((i+1, raw))
        continue

    # 3. Línea que comienza con espacio
    if raw.startswith(" "):
        broken.append((i+1, raw))
        continue

    # 4. Línea sin comas
    if "," not in raw:
        broken.append((i+1, raw))
        continue

    # 5. Línea cuyo primer carácter NO es un número
    first_char = stripped[0]
    if not first_char.isdigit():
        broken.append((i+1, raw))
        continue

print("Líneas sospechosas:")
for idx, content in broken:
    print(f" - Línea {idx}: {content}")


In [ ]:
#Unir los registros partidos
import csv

csv_file = "/content/drive/MyDrive/Documentos/Pablo/Proyectos/Datos abiertos/Datos crudos/Datos Limpios/SIV Señal Vertical.csv"
output_file = "data_reconstruido.csv"

with open(csv_file, encoding="utf-8") as f:
    lines = f.readlines()

reconstructed = []
current_row = None

def is_continuation(line, is_header=False):
    raw = line.rstrip("\n")
    stripped = raw.strip()

    # No aplicar reglas a encabezado
    if is_header:
        return False

    # --- REGLAS DE DETECCIÓN DE CONTINUACIÓN ---
    if stripped == "":
        return True

    if stripped.startswith(","):
        return True

    if raw.startswith(" "):
        return True

    if "," not in raw:
        return True

    if not stripped[0].isdigit():  # NO empieza con número
        return True

    # Si pasa todo, es una línea válida
    return False


for i, line in enumerate(lines):
    is_header = (i == 0)
    raw = line.rstrip("\n")

    if is_continuation(raw, is_header=is_header):
        # Esta línea es continuación → concatenarla
        if current_row is None:
            # Si por alguna razón aparece un fragmento antes de cualquier registro válido
            current_row = raw
        else:
            current_row += " " + raw.strip()
    else:
        # Esta línea es un registro válido → empezar uno nuevo
        if current_row is not None:
            reconstructed.append(current_row)
        current_row = raw  # iniciar un nuevo registro

# Agregar el último registro si existe
if current_row is not None:
    reconstructed.append(current_row)


# --- Guardar resultado reconstruido ---
with open(output_file, "w", encoding="utf-8", newline="") as f:
    for row in reconstructed:
        f.write(row + "\n")

print(f"Reconstrucción completa. Total registros: {len(reconstructed)}")
print(f"Archivo generado: {output_file}")


In [ ]:
import csv

input_file = "/content/data_reconstruido.csv"
output_file = "SIV Señal Vertical.csv"

clean_rows = []

with open(input_file, encoding="utf-8") as f:
    reader = csv.reader(f)
    rows = list(reader)

# Número correcto de columnas según encabezado
expected_cols = len(rows[0])

# Índice de la columna de texto (detección automática opcional)
# → Si ya sabes cuál es, ponlo manual.
# Aquí asumo que el texto es la ÚLTIMA columna ANTES de donde empiezan a romperse las filas.
# Si prefieres indicar un índice específico, me lo dices.
text_col_index = expected_cols - 10   # Ajusta si el texto está en otra posición

for row in rows:
    # Caso perfecto: longitud correcta
    if len(row) == expected_cols:
        clean_rows.append(row)
        continue

    # Caso con columnas de más → texto roto por comas internas
    if len(row) > expected_cols:
        # Tomamos las columnas buenas iniciales
        fixed = row[:expected_cols]

        # Partes extra que son fragmentos del texto roto
        extra = row[expected_cols - 1 :]

        # Unir fragmentos del texto, quitando comas internas
        merged_text = " ".join(extra).replace(",", " ")

        # Insertar texto limpio en la columna correcta
        fixed[text_col_index] = merged_text

        clean_rows.append(fixed)
        continue

    # Si tiene menos columnas (casos raros):
    # Aquí puedes mejorar según tu caso.
    clean_rows.append(row)


# Guardar archivo limpio
with open(output_file, "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(clean_rows)

print("Archivo corregido:", output_file)
print("Columnas esperadas:", expected_cols)


In [ ]:
import csv

# =============================
#  CONFIG
# =============================
input_csv = "/content/SIV Señal Vertical.csv"
temp_reconstructed = "/content/data_reconstruido.csv"
output_clean = "/content/SIV Señal Vertical_Limpio.csv"

# Índice donde está tu columna de texto (0 = primera columna)
text_col_index = -10   # AJUSTA según tu archivo

# =============================
#  1. LEER CSV ORIGINAL
# =============================
with open(input_csv, encoding="utf-8") as f:
    lines = f.readlines()

# =============================
#  2. DETECTAR Y RECONSTRUIR REGISTROS PARTIDOS
# =============================

def is_continuation(line, is_header=False):
    raw = line.rstrip("\n")
    stripped = raw.strip()

    if is_header:
        return False

    # Reglas de que es una continuación
    if stripped == "":
        return True
    if stripped.startswith(","):
        return True
    if raw.startswith(" "):
        return True
    if "," not in raw:
        return True
    if not stripped[0].isdigit():
        return True

    return False


reconstructed = []
current_row = None

for i, line in enumerate(lines):
    is_header = (i == 0)
    raw = line.rstrip("\n")

    if is_continuation(raw, is_header=is_header):
        if current_row is None:
            current_row = raw
        else:
            current_row += " " + raw.strip()
    else:
        if current_row is not None:
            reconstructed.append(current_row)
        current_row = raw

if current_row is not None:
    reconstructed.append(current_row)

# Guardar reconstrucción temporal
with open(temp_reconstructed, "w", encoding="utf-8", newline="") as f:
    for row in reconstructed:
        f.write(row + "\n")

print(f"✔ Registros reconstruidos: {len(reconstructed)}\nArchivo: {temp_reconstructed}")


# =============================
#  3. CORREGIR COMAS INTERNAS EN COLUMNAS
# =============================
with open(temp_reconstructed, encoding="utf-8") as f:
    reader = csv.reader(f)
    rows = list(reader)

expected_cols = len(rows[0])  # columnas esperadas según encabezado
clean_rows = []

for row in rows:
    if len(row) == expected_cols:
        clean_rows.append(row)
        continue

    if len(row) > expected_cols:
        fixed = row[:expected_cols]
        extra = row[expected_cols - 1:]
        merged_text = " ".join(extra).replace(",", " ")
        fixed[text_col_index] = merged_text
        clean_rows.append(fixed)
        continue

    clean_rows.append(row)


# =============================
#  4. GUARDAR RESULTADO FINAL
# =============================
with open(output_clean, "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(clean_rows)

print("\n==============================")
print("✔ ARCHIVO FINAL GENERADO")
print("==============================")
print("Archivo limpio:", output_clean)
print("Columnas esperadas:", expected_cols)
